# Bin Packing Problem

### Useful packages

In [432]:
import numpy as np
import random
from scipy.optimize import milp, LinearConstraint

### Instantiate data

Create VMs with random vcpu, memory and disk requirements.



In [433]:
def generate_vm_name():
    adjectives = ["Wobbly", "Fluffy", "Spicy", "Grumpy",
                  "Sneaky", "Jolly", "Zesty", "Quirky", "Bouncy", "Soggy"]
    nouns = ["Penguin", "Toaster", "Banana", "Octopus", "Pancake",
             "Noodle", "Cactus", "Sasquatch", "Pickle", "Muffin"]
    number = random.randint(1, 999)
    return f"{random.choice(adjectives)}{random.choice(nouns)}-{number}"


def generate_vm(n: int, max_vcpu=8, max_memory=24, max_disk_space=32) -> list :
    """"
    Generate a list of VMs using random features

    Input:
    n: int - number of VMs to generate
    """
    res = []

    for i in range(n):
        # generate VM name
        vm_name = generate_vm_name()
        # generate vCPU
        vcpu = random.randint(1, max_vcpu)
        # generate Memory (GB)
        memory = random.randint(1, max_memory)
        # generate Disk Space (GB)
        disk_space = random.randint(1, max_disk_space)
        res.append([vm_name, vcpu, memory, disk_space])
    # type cast to numpy array for better performance
    return np.array(res)

def show_vms(vms: list):
    max_len_name = max([len(vm[0]) for vm in vms]) + 5
    print(f"VM Name{' '*(max_len_name-7)}vCPU\tMemory(GB)\tDisk Space(GB)")
    for vm in vms:
        print(f"{vm[0]+ ' '*(max_len_name - len(vm[0]))}\t{vm[1]}\t{vm[2]}\t\t{vm[3]}")

vms = generate_vm(10)
show_vms(vms)

VM Name                 vCPU	Memory(GB)	Disk Space(GB)
FluffyToaster-708       	1	24		21
BouncyMuffin-691        	7	15		21
FluffyPancake-174       	5	9		32
SpicyToaster-434        	6	4		32
BouncyPenguin-903       	5	22		14
ZestyPickle-890         	3	11		11
QuirkyPickle-551        	2	2		9
SpicyBanana-557         	3	6		5
WobblySasquatch-961     	7	16		4
FluffyMuffin-527        	7	15		25


In [434]:
vms[:,1].dtype

dtype('<U21')

In [435]:
def generate_cluster_name():
    zones = ["us-west1", "us-west2", "us-west3", "us-east1", "us-east2", "us-east3"]
    names = ["Alpha", "Beta", "Gamma", "Delta", "Epsilon", "Zeta", "Eta", "Theta", "Iota", "Kappa"]
    number = random.randint(1, 999)
    return f"{random.choice(zones)}-{random.choice(names)}-{number}"

def generate_cluster(n: int, max_vcpu=64, max_memory=64, max_disk_space=128) -> list:
    res = []

    for i in range(n):
        # generate cluster name
        cluster_name = generate_cluster_name()
        # generate vCPU
        vcpu = random.randint(1, max_vcpu)
        # generate Memory (GB)
        memory = random.randint(1, max_memory)
        # generate Disk Space (GB)
        disk_space = random.randint(1, max_disk_space)
        res.append([cluster_name, vcpu, memory, disk_space])
    return np.array(res)

def show_clusters(clusters: list):
    max_len_name = max([len(cluster[0]) for cluster in clusters]) + 5
    print(f"Cluster Name{' '*(max_len_name-12)}\tvCPU\tMemory(GB)\tDisk Space(GB)")
    for cluster in clusters:
        print(f"{cluster[0]+ ' '*(max_len_name - len(cluster[0]))}\t{cluster[1]}\t{cluster[2]}\t\t{cluster[3]}")

clusters = generate_cluster(10)
show_clusters(clusters)

Cluster Name           	vCPU	Memory(GB)	Disk Space(GB)
us-west1-Zeta-864      	1	8		53
us-east3-Alpha-238     	9	27		11
us-east1-Iota-619      	54	21		82
us-west1-Gamma-943     	58	4		90
us-west1-Zeta-431      	47	43		67
us-west2-Gamma-887     	56	33		13
us-east1-Beta-91       	11	7		50
us-west2-Eta-739       	29	14		93
us-west2-Eta-771       	10	40		74
us-east2-Eta-567       	53	17		73


$ \min \sum_{i=1}^{n} xy_i $

s.t.$ \sum_{j=1}^{m} w_j x_{ij} \leq c_i, \forall i \in \{1, \ldots, n\} $

$ \sum_{i=1}^{n} x_{ij} = 1, \forall j \in \{1, \ldots, m\} $

$y_i \in \{0, 1\}, \forall i \in \{1, \ldots, n\}$

$x_{ij} \in \{0, 1\}, \forall i \in \{1, \ldots, n\}, j \in \{1, \ldots, m\}$

Idea create a big variable 

[cluster0, cluster1, ... clusterN, 

vm0_cluster0, vm0_cluster1, ... vm0_clusterN, 

vm1_cluster0, vm1_cluster1, ... vm1_clusterN, 

... 

vmM_cluster0, vmM_cluster1, ... vmM_clusterN]

### Define the relaxed model

In [436]:
def one_relaxed_constraint_A(clusters, vms, constraint_idx):
    """
    A helper function to generate the constraint matrix for a single constraint
    
    Input:
    clusters: np.array(str, float, float, float) - a list of clusters with each row representing a cluster and each column representing a resource
    vms: np.array(str, float, float, float) - a list of VMs with each row representing a VM and each column representing a resource
    constraint_idx: int - the index of the constraint to generate the matrix for

    Output:
    A: np.array(float) - the constraint matrix for the given constraint_idx
    """


    cluster_constraints = clusters[:, constraint_idx].astype(float)
    vm_constraints = vms[:, constraint_idx].astype(float)
    n_clusters = len(cluster_constraints)
    n_vms = len(vm_constraints)
    A = np.zeros((n_clusters, n_clusters + n_clusters*n_vms))
    for i in range(n_clusters):
        A[i, i] = -cluster_constraints[i]
        A[i, n_clusters:][[i + j * n_clusters for j in range(n_clusters)]] = vm_constraints
    return A


def relaxed_bin_packing(clusters, vms):
    """
    Relaxed big packing problem:
    Given a set of clusters and a set of VMs, assign a COEFFICIENT for each VM to a cluster such that
    the total resource usage of each cluster does not exceed the capacity of the cluster.
    The objective is to minimize the number of clusters used.

    Input:
    clusters: np.array(str, float, float, float) - a list of clusters with each row representing a cluster and each column representing a resource
    vms: np.array(str, float, float, float) - a list of VMs with each row representing a VM and each column representing a resource

    Output:
    cluster_assignment: np.array(int) - a list of cluster assignments for each VM
    vm_assignment: np.array(float) - a list of coefficients for each VM to each cluster
    """

    # Define sizes
    n_cluster = len(clusters)
    n_vm = len(vms)
    n_constraints = vms.shape[1] - 1 # remove the name column
    n_var = n_cluster + n_cluster*n_vm

    # Define the objective function
    c = np.zeros(n_var)
    c[:n_cluster] = 1

    # Define the constraints
    # constraints on the variables
    A = np.zeros((n_var + n_vm, n_var))
    A[:n_var] = np.eye(n_var)

    # constraints on the coefficients of the VMs
    for i in range(n_vm):
        A[n_var + i, n_cluster + i*n_cluster:n_cluster + (i+1)*n_cluster] = np.ones(n_cluster)

    # constraints on the cluster capacities
    for i in range(n_constraints):
        A = np.concatenate((A, one_relaxed_constraint_A(clusters, vms, i+1)))
    # A shape = (n_var + n_vm + n_constraints*n_cluster, n_var)

    # Define the upper and lower bounds
    ub = np.concatenate((np.ones(n_var + n_vm), np.zeros(n_cluster*n_constraints)))
    lb = np.concatenate((np.zeros(n_var), np.ones(n_vm), [-np.inf]*(n_cluster*n_constraints)))

    # Define the integrality
    integrality = np.array([True]*n_cluster + [False]*n_cluster*n_vm)

    # Define the constraints as a LinearConstraint object
    constraints = LinearConstraint(A, lb, ub)

    # Solve the problem
    res = milp(c=c, constraints=constraints, integrality=integrality)

    # Return the results (cluster assignment, vm assignment)
    return res.x[:n_cluster], res.x[n_cluster:].reshape(n_vm, n_cluster)


assign_cluster, assign_vm = relaxed_bin_packing(clusters, vms)

In [437]:
def pretty_assignment_print(assign_cluster, assign_vm, clusters=clusters, vms=vms):
    n_cluster = len(assign_cluster)
    n_vm = len(assign_vm)
    max_len_cluster = max([len(cluster[0]) for cluster in clusters]) + 5
    print(
        f"Cluster Name{' '*(max_len_cluster-12)}\tCPU usage (%)\tMem usage (%)\tDisk usage (%)\tAssigned VMs")
    for i in range(n_cluster):
        assigned_vms = [vms[j][0] for j in range(n_vm) if assign_vm[j][i] > 0]
        cpu_usage = sum([float(vms[j][1])*assign_vm[j][i]
                        for j in range(n_vm)])
        memory_usage = sum([float(vms[j][2])*assign_vm[j][i]
                           for j in range(n_vm)])
        disk_space_usage = sum(
            [float(vms[j][3])*assign_vm[j][i] for j in range(n_vm)])
        print(f"{clusters[i][0] + ' '*(max_len_cluster - len(clusters[i][0]))}\t{cpu_usage/float(clusters[i][1])*100:.2f}%\t\t{memory_usage/float(clusters[i][2])*100:.2f}%\t\t{disk_space_usage/float(clusters[i][3])*100:.2f}%\t\t{len(assigned_vms)}: {', '.join(assigned_vms)}")


pretty_assignment_print(assign_cluster, assign_vm)

Cluster Name           	CPU usage (%)	Mem usage (%)	Disk usage (%)	Assigned VMs
us-west1-Zeta-864      	0.00%		0.00%		0.00%		0: 
us-east3-Alpha-238     	0.00%		0.00%		0.00%		0: 
us-east1-Iota-619      	11.67%		73.81%		40.10%		3: FluffyToaster-708, SpicyToaster-434, ZestyPickle-890
us-west1-Gamma-943     	0.00%		0.00%		0.00%		0: 
us-west1-Zeta-431      	43.70%		100.00%		100.00%		6: BouncyMuffin-691, SpicyToaster-434, BouncyPenguin-903, ZestyPickle-890, SpicyBanana-557, FluffyMuffin-527
us-west2-Gamma-887     	16.36%		77.27%		77.27%		2: BouncyPenguin-903, WobblySasquatch-961
us-east1-Beta-91       	0.00%		0.00%		0.00%		0: 
us-west2-Eta-739       	0.00%		0.00%		0.00%		0: 
us-west2-Eta-771       	100.00%		100.00%		86.58%		4: FluffyToaster-708, FluffyPancake-174, BouncyPenguin-903, QuirkyPickle-551
us-east2-Eta-567       	0.00%		0.00%		0.00%		0: 


### Define the model

In [438]:
def bin_packing(clusters, vms):
    """ 
    Bin packing problem:
    Given a set of clusters and a set of VMs, assign each VM to a cluster such that
    the total resource usage of each cluster does not exceed the capacity of the cluster.
    
    Input:
    clusters: np.array(str, float, float, float) - a list of clusters with each row representing a cluster and each column representing a resource
    vms: np.array(str, float, float, float) - a list of VMs with each row representing a VM and each column representing a resource

    Output:
    cluster_assignment: np.array(int) - a list of cluster assignments for each VM
    vms_assignment: np.array(int) - a list of VM assignments to each cluster
    """

   
    # Define sizes
    n_cluster = len(clusters)
    n_vm = len(vms)
    n_constraints = vms.shape[1] - 1 # remove the name column
    n_var = n_cluster + n_cluster*n_vm

    # Define the objective function
    c = np.zeros(n_var)
    c[:n_cluster] = 1

    # Define the constraints
    # constraints on the variables
    A = np.zeros((n_var + n_vm, n_var))
    A[:n_var] = np.eye(n_var)

    # constraints on the coefficients of the VMs
    for i in range(n_vm):
        A[n_var + i, n_cluster + i*n_cluster:n_cluster + (i+1)*n_cluster] = np.ones(n_cluster)

    # constraints on the cluster capacities
    for i in range(n_constraints):
        A = np.concatenate((A, one_relaxed_constraint_A(clusters, vms, i+1)))
    # A shape = (n_var + n_vm + n_constraints*n_cluster, n_var)

    # Define the upper and lower bounds
    ub = np.concatenate((np.ones(n_var + n_vm), np.zeros(n_cluster*n_constraints)))
    lb = np.concatenate((np.zeros(n_var), np.ones(n_vm), [-np.inf]*(n_cluster*n_constraints)))

    # Define the integrality
    integrality = np.array([True]*n_var)

    # Define the constraints as a LinearConstraint object
    constraints = LinearConstraint(A, lb, ub)

    # Solve the problem
    res = milp(c=c, constraints=constraints, integrality=integrality)

    # Return the results (cluster assignment, vm assignment)
    return res.x[:n_cluster], res.x[n_cluster:].reshape(n_vm, n_cluster)


assign_cluster, assign_vm = bin_packing(clusters, vms)

In [439]:
pretty_assignment_print(assign_cluster, assign_vm)

Cluster Name           	CPU usage (%)	Mem usage (%)	Disk usage (%)	Assigned VMs
us-west1-Zeta-864      	0.00%		0.00%		0.00%		0: 
us-east3-Alpha-238     	0.00%		0.00%		0.00%		0: 
us-east1-Iota-619      	27.78%		100.00%		80.49%		3: SpicyToaster-434, QuirkyPickle-551, FluffyMuffin-527
us-west1-Gamma-943     	0.00%		0.00%		0.00%		0: 
us-west1-Zeta-431      	27.66%		97.67%		85.07%		3: FluffyPancake-174, BouncyPenguin-903, ZestyPickle-890
us-west2-Gamma-887     	17.86%		66.67%		69.23%		2: SpicyBanana-557, WobblySasquatch-961
us-east1-Beta-91       	0.00%		0.00%		0.00%		0: 
us-west2-Eta-739       	0.00%		0.00%		0.00%		0: 
us-west2-Eta-771       	80.00%		97.50%		56.76%		2: FluffyToaster-708, BouncyMuffin-691
us-east2-Eta-567       	0.00%		0.00%		0.00%		0: 


### 1. Affinity rules between some set of VMs

some VMs could share a cluster / some others couln't (affinity / anti-affinity)

### 2. All servers are partly occupied vs totally empty and all with the same characteristics

1 type of server vs multiple types of several type

### 3. VMs could be splitted over several servers

VMs could be splitted over several servers

### 4. Consider VMs families, each family is given a criticity level between 1 to 3

In [440]:
# # simple integer programming with SciPY

# # define the objective function
# c = np.array([1, 1, 0, 0, 0, 0]) # 2 clusters, 2 VMs

# # define the constraints
# A = np.array([[1, 0,     0, 0, 0, 0],
#               [0, 1,     0, 0, 0, 0],
#               [0, 0,     1, 0, 0, 0],
#               [0, 0,     0, 1, 0, 0],
#               [0, 0,     0, 0, 1, 0],
#               [0, 0,    0, 0, 0, 1], # n_cluster + n_vm*n_cluster

#               [-3, 0,    2, 0, 3, 0],
#               [0, -3,    0, 2, 0, 3], # n_cluster

#               [0, 0,     1, 1, 0, 0],
#               [0, 0,     0, 0, 1, 1]]) # n_vm
#             # n_cluster   n_cluster * n_vm

# # 0 <= y1 <= 1
# # 0 <= y2 <= 1
# # 2x00 + 3x01 <= 4y1
# # 2x10 + 3x11 <= 5y2
# # x00 + x01 <= 1
# # x10 <= 1
# # x11 <= 1

# ub = np.array([1, 1, 1, 1, 1, 1, 0, 0, 1, 1]) # upper bounds
# lb = np.array([0, 0, 0, 0, 0, 0, -np.inf, -np.inf, 1, 1]) # lower bounds
# # x00+ x01 = 1
# # x10 + x11 = 1
# #constraints1 = {'type': 'eq', 'fun': lambda x: np.array([x[2] + x[3] - 1, x[4] + x[5] - 1])}
# constraints = LinearConstraint(A, lb, ub)

# integrality = np.array([True, True, False, False, False, False])

# res = milp(c=c, constraints=constraints, integrality=integrality)
# res.x



# def one_constraints_bin_packing(clusters, vms, constraint_idx):
#     cluster_constraints = clusters[:, constraint_idx].astype(float)
#     vm_constraints = vms[:, constraint_idx].astype(float)
#     n_clusters = len(cluster_constraints)
#     n_vms = len(vm_constraints)

#     n_var  = n_clusters + n_vms*n_clusters
#     c = np.zeros(n_var)
#     c[:n_clusters] = 1

#     A = np.zeros((n_var + n_clusters + n_vms, n_var))

#     A[:n_var] = np.eye(n_var)

#     start = n_var
#     A[start:start+n_clusters] = one_constraint_A(clusters, vms, constraint_idx)

#     start += n_clusters
#     for i in range(n_vms):
#         A[start + i, n_clusters + i*n_clusters:n_clusters + (i+1)*n_clusters] = np.ones(n_clusters)

#     ub =np.concatenate((np.ones(n_var), np.zeros(n_clusters), np.ones(n_vms)))
#     lb = np.concatenate((np.zeros(n_var), [-np.inf]*n_clusters, np.ones(n_vms)))

#     integrality = np.array([True]*n_clusters + [False]*n_clusters*n_vms)
#     constraints = LinearConstraint(A, lb, ub)

#     res = milp(c=c, constraints=constraints, integrality=integrality)
#     return res.x[:n_clusters], res.x[n_clusters:].reshape(n_vms, n_clusters)

# one_constraints_bin_packing(np.array(clusters), np.array(vms), 2)